In [13]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.svm import SVC
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import copy

from quask.core import Ansatz, Kernel, KernelFactory, KernelType
from quask.core_implementation import QiskitKernel
from quask.optimizer.base_kernel_optimizer import BaseKernelOptimizer
from quask.evaluator import CenteredKernelAlignmentEvaluator
from quask.optimizer.bayesian_optimizer import BayesianOptimizer

def create_qiskit_noiseless(ansatz: Ansatz, measurement: str, type: KernelType):
    return QiskitKernel(ansatz, measurement, type, n_shots=None)

KernelFactory.add_implementation('qiskit_noiseless2', create_qiskit_noiseless)
KernelFactory.set_current_implementation('qiskit_noiseless2')

In [14]:
class RandomOptimizer(BaseKernelOptimizer):

    def __init__(self, initial_kernel, X, y, ke):
        super().__init__(initial_kernel, X, y, ke)

    def optimize(self):
        kernel = copy.deepcopy(self.initial_kernel)
        cost = self.ke.evaluate(kernel, None, self.X, self.y)
        N_TENTATIVES = 10
        for i in range(N_TENTATIVES):
            new_kernel = copy.deepcopy(kernel)
            i_operation = np.random.randint(new_kernel.ansatz.n_operations)
            i_feature = np.random.randint(new_kernel.ansatz.n_features)
            i_wires = np.random.choice(range(new_kernel.ansatz.n_qubits), 2, replace=False).tolist()
            i_gen = np.random.choice(['I', 'Z', 'X', 'Y'], 2, replace=True)
            i_gen = "".join(i_gen.tolist())
            i_bandwidth = np.random.rand()
            new_kernel.ansatz.change_operation(i_operation, i_feature, i_wires, i_gen, i_bandwidth)
            new_cost = self.ke.evaluate(new_kernel, None, self.X, self.y)
            print("Cost of the new solution:", new_cost)
            if cost > new_cost:
                kernel = new_kernel
                cost = new_cost
        return kernel

In [25]:
N_FEATURES   = 4
N_OPERATIONS = N_FEATURES   # ← was 5, now 4
N_QUBITS     = 4

ansatz = Ansatz(
    n_features  = N_FEATURES,
    n_qubits    = N_QUBITS,
    n_operations= N_OPERATIONS
)
ansatz.initialize_to_identity()

# now each op.feature will be in [0,1,2,3], no IndexError
print([op.feature for op in ansatz.operation_list])

kernel = KernelFactory.create_kernel(ansatz, "Z" * N_QUBITS, KernelType.FIDELITY)

N_ELEMENTS_PER_CLASS = 20
iris = load_iris()
X = np.vstack([iris.data[0:N_ELEMENTS_PER_CLASS], iris.data[50:50+N_ELEMENTS_PER_CLASS]])
y = np.array([-1] * N_ELEMENTS_PER_CLASS + [1] * N_ELEMENTS_PER_CLASS)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=5454)
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
ce = CenteredKernelAlignmentEvaluator()

[4, 4, 4, 4]


In [39]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import quask
from quask.core import Ansatz, KernelFactory, KernelType
from quask.core_implementation import QiskitKernel
from quask.evaluator import CenteredKernelAlignmentEvaluator
from quask.optimizer.bayesian_optimizer import BayesianOptimizer

# 1) Load & split Iris (only two classes, 20 samples each)
iris = load_iris()
X = np.vstack([iris.data[:20], iris.data[50:70]])
y = np.array([-1]*20 + [1]*20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.5, random_state=5454
)
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test  = scaler.transform(X_test)

# 2) Build an identity Ansatz with one op per feature
N_FEATURES   = X_train.shape[1]   # 4
N_OPERATIONS = N_FEATURES        # ← must match to avoid OOB indexing
N_QUBITS     = 4

ansatz = Ansatz(
    n_features   = N_FEATURES,
    n_qubits     = N_QUBITS,
    n_operations = N_OPERATIONS
)
ansatz.initialize_to_identity()

print("Operation ⟨feature⟩ indices:", [op.feature for op in ansatz.operation_list])
# should print [0,1,2,3]

# 3) Register the “noiseless” Qiskit backend
def create_qiskit_noiseless(ansatz, measurement: str, type: KernelType):
    # Here platform="infty_shots" forces Estimator/Aer path inside QiskitKernel
    return QiskitKernel(
        ansatz,
        measurement,
        type,
        platform="infty_shots",
        n_shots=None  # None means no sampling, use statevector/estimator
    )

# KernelFactory.add_implementation('qiskit_noiseless5', create_qiskit_noiseless)
KernelFactory.set_current_implementation('qiskit_noiseless5')

kernel = KernelFactory.create_kernel(
    ansatz,
    measurement = "Z" * N_QUBITS,
    type        = KernelType.FIDELITY
)

# 4) Evaluate initial centered‐alignment cost
ce = CenteredKernelAlignmentEvaluator()
init_cost = ce.evaluate(kernel, None, X_train, y_train)
print("The initial cost is:", init_cost)

# 5) Run Bayesian optimization (2 epochs, 1 point per epoch)
optimizer = BayesianOptimizer(kernel, X_train, y_train, ce)
opt_kernel = optimizer.optimize(n_epochs=2, n_points=1, n_jobs=1)

final_cost = ce.evaluate(opt_kernel, None, X_train, y_train)
print("The final cost is:", final_cost)

# 6) Train & test an SVM with the optimized kernel
model   = SVC(kernel='precomputed')
K_train = opt_kernel.build_kernel(X_train, X_train)
model.fit(K_train, y_train)

K_test  = opt_kernel.build_kernel(X_test, X_train, matrix="test")
y_pred  = model.predict(K_test)

accuracy = np.mean(y_pred == y_test)
print("Accuracy:", accuracy)


Operation ⟨feature⟩ indices: [4, 4, 4, 4]


IndexError: list index out of range

In [26]:
# right after you do
ansatz = Ansatz(n_features=N_FEATURES,
                n_qubits=N_QUBITS,
                n_operations=N_OPERATIONS)
ansatz.initialize_to_identity()

# add this:
print("→ n_features   =", ansatz.n_features)
print("→ n_operations =", ansatz.n_operations)
print("→ op.feature   =", [op.feature for op in ansatz.operation_list])


→ n_features   = 4
→ n_operations = 4
→ op.feature   = [4, 4, 4, 4]


In [27]:
print("The initial cost is:", ce.evaluate(kernel, None, X_train, y_train))
optimizer = BayesianOptimizer(kernel, X_train, y_train, ce)
optimized_kernel = optimizer.optimize(n_epochs=2, n_points=1, n_jobs=1)
print("The final cost is:", ce.evaluate(optimized_kernel, None, X_train, y_train))

IndexError: list index out of range

In [ ]:
model = SVC(kernel='precomputed')
K_train = optimized_kernel.build_kernel(X_train, X_train)
model.fit(K_train, y_train)
K_test = optimized_kernel.build_kernel(X_test, X_train)
y_pred = model.predict(K_test)
accuracy = np.sum(y_test == y_pred) / len(y_test)
print("Accuracy:", accuracy)

NameError: name 'optimized_kernel' is not defined